In [1]:
#create dataset
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)



In [2]:
#load dataset
from mxnet.gluon import data as gdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)


In [3]:
for X, y in data_iter:
    print(X, y)
    break




[[ 0.9632486  -1.1641017 ]
 [ 0.5299177   0.64164335]
 [-0.5371306   1.5127277 ]
 [-2.0051894  -1.0490868 ]
 [ 1.2391903  -0.5411598 ]
 [ 0.72018015  0.9385755 ]
 [ 0.79127795  0.69427365]
 [ 0.30487114 -0.2350973 ]
 [-1.4493372  -0.4517469 ]
 [-1.4933364  -2.392217  ]]
<NDArray 10x2 @cpu(0)> 
[10.080251   3.0920963 -2.015738   3.7439334  8.525438   2.4503958
  3.423368   5.609713   2.8295393  9.34088  ]
<NDArray 10 @cpu(0)>


In [4]:
#define model
from mxnet.gluon import nn
#Sequential in many scenes
net = nn.Sequential()
#add Dense layer 
net.add(nn.Dense(1))

#init the model parameters
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

#define loss function
from mxnet.gluon import loss as gloss
#L2 Euclidean norm 
loss = gloss.L2Loss()  # 平方损失又称 L2范数损失

#define optimize algorithm function
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})



In [5]:
# 3 epochs
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))




epoch 1, loss: 0.034808
epoch 2, loss: 0.000124
epoch 3, loss: 0.000049


In [6]:
dense = net[0]
true_w, dense.weight.data()



([2, -3.4],
 
 [[ 1.9991777 -3.3997512]]
 <NDArray 1x2 @cpu(0)>)

In [7]:
true_b, dense.bias.data()



(4.2,
 
 [4.2000227]
 <NDArray 1 @cpu(0)>)

In [8]:
#use the gluon we can  implement the linear regression model simplely
print(loss(net(X),y),loss(net(X),y).mean())




[6.0533195e-05 1.9712042e-04 1.7850432e-08 6.9471571e-07 1.4990978e-06
 9.0498113e-05 1.6368176e-05 1.9900441e-05 1.8250199e-04 3.3255528e-06]
<NDArray 10 @cpu(0)> 
[5.7245954e-05]
<NDArray 1 @cpu(0)>


In [13]:
#when use loss().mean() you'ld better descent the trainer().step


UserWarning: Gradient of Parameter `dense0_weight` on context cpu(0) has not been updated by backward since last `step`. This could mean a bug in your model that made it only use a subset of the Parameters (Blocks) for this iteration. If you are intentionally only using a subset, call step with ignore_stale_grad=True to suppress this warning and skip updating of Parameters with stale gradient